In [1]:
import nltk
import argparse
import os
import sys,re
import pickle
import pandas as pd
from nltk.corpus import stopwords

In [2]:
docs_path='/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/'
data_path='/Users/rohan/Desktop/6th sem/Information Retrieval/other/Data/'

In [3]:
"Preprocessing Text"
def preprocess_text(text):
    processed_text = text.lower()
    processed_text = processed_text.replace("’", "'")
    processed_text = processed_text.replace("“", '"')
    processed_text = processed_text.replace("”", '"')
    non_words = re.compile(r"[^A-Za-z']+")
    processed_text = re.sub(non_words, ' ', processed_text)

    return processed_text

In [4]:
def get_text_from_file(filename):
    with open(filename, encoding='cp1252', mode='r') as f:
        text = f.read()
    return text

In [5]:
def get_words_from_text(text):
    stop_words = set(stopwords.words('english'))
    processed_text = preprocess_text(text)
    words = {w for w in processed_text.split() if w not in stop_words}#Removing Stop Words
    wordset=[]
    wordset1=[]
    #Stemming
    porter = nltk.PorterStemmer()
    for word1 in words:
        words=porter.stem(word1)
        wordset.append(words)
    #Lemmatizing
    wnl = nltk.WordNetLemmatizer()
    for word2 in wordset:
        words1=wnl.lemmatize(word2)
        wordset1.append(words1)    
    
    return wordset1

In [6]:
def build_inverted_index(docs_path):
    inverted_index = {}

    for doc_file in os.listdir(docs_path):
        filename = os.path.join(docs_path, doc_file)
        text = get_text_from_file(filename)
        words = get_words_from_text(text)

        for word in words:
            if inverted_index.get(word, None) is None:
                inverted_index[word] = {filename}
            else:
                inverted_index[word].add(filename)
    return inverted_index

In [7]:
def index(docs_path, data_path):
    inverted_index = build_inverted_index(docs_path)
    dic_file = os.path.join(data_path, 'dictionary.txt')
    inverted_index_file = os.path.join(data_path, 'inverted_index.pickle')

    with open(dic_file, mode='w') as f:
        for word in inverted_index.keys():
            f.write(word + '\n')

    with open(inverted_index_file, mode='wb') as f:
        pickle.dump(inverted_index, f)

In [8]:
index(docs_path, data_path)

In [9]:
df=pd.read_pickle('/Users/rohan/Desktop/6th sem/Information Retrieval/other/Data/inverted_index.pickle')

In [10]:
df2 = pd.DataFrame.from_dict(df, orient="index")

In [11]:
df2.to_csv('/Users/rohan/Desktop/6th sem/Information Retrieval/other/Data/inverted_index.csv')

In [12]:
#  "Querying AND , OR , NOT"
inverted_index_file = os.path.join(os.getcwd(), 'Data', 'inverted_index.pickle')

with open(inverted_index_file, mode='rb') as f:
    inverted_index = pickle.load(f)

dictionary = inverted_index.keys()

non_words = re.compile(r"[^A-Za-z'?]+")
stop_words = set(stopwords.words('english'))

In [29]:
#Enter your query with space in between and also boolean operation "AND,OR,NOT"
query="trader"
operation="not"

In [30]:
# Remove all stopwords and words which is not in dictionary
query = query.lower()
operation=operation.lower()
query = re.sub(non_words, ' ', query)
words = {word for word in query.split() 
         if word not in stop_words and word in dictionary}

list_docs=set()
for doc_file in os.listdir(docs_path):
    filename = os.path.join(docs_path, doc_file)
    list_docs.add(filename)

In [31]:
words_list=query.split()

In [32]:
if operation=="and":
    result = inverted_index.get(words_list[0])
    result1=inverted_index.get(words_list[1])
    output=result.intersection(result1)
elif operation=="or":
    result = inverted_index.get(words_list[0])
    result1=inverted_index.get(words_list[1])
    output=result.union(result1)
elif operation=="not":
    result = inverted_index.get(words_list[0])
    output=list_docs.difference(result)
print(output)

{'/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/Rout of hossars.txt', '/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/yoked with an unbeliever.txt', '/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/False Dawn.txt', '/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/Wressley of the foreign office.txt', '/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/.DS_Store', '/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/Consequences.txt', '/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/Cupids arrow.txt', '/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/on the strength of a likeness.txt', '/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/pig.txt', '/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/Miss Yougals sais.txt', '/Users/rohan/Desktop/6th sem/Information Retrieval/other/Docs/In the pride of his youth.txt', '/Users/rohan/Desktop/6th sem/Information Retri

In [ ]:
result1=None
result=None
for word in words:
    if result is None:
        result = inverted_index.get(word)
    elif operation=="and":
        print('and')
        result.intersection_update(inverted_index.get(word))
    elif operation=="or":
        print('or')
        result.union(inverted_index.get(word))
    elif operation=="not":
        b=inverted_index.get(word)
        result1=list_docs.difference(b)
        print(result1)
    else:
        print(result)
print(result)

In [ ]:

list_a=set()
for word in words:
    b=inverted_index.get(word)
    #list_a.add(b)
#print(type(list_a))
#list_b={*list_a}
result1=list_docs.difference(b)
print(result1)

In [ ]:
result=None
result = inverted_index.get('lie')
print(result)

In [ ]:
result1=None
result = inverted_index.get('lie')
result1=inverted_index.get('unhappi')
print(result1)

In [ ]:
result.union(result1)

In [ ]:
list_docs.difference(result1)

In [ ]:
str1="vatican and teacher or start"
w=str1.split()


In [ ]:
i=-1
result=None
for s in w:
    i=i+1
    if s=="and" :
        x=i-1;
        y=i+1;
        word1=inverted_index.get(w[x])
        word2=inverted_index.get(w[y])
        result=word1.intersection(word2)
    elif s=="and" and i>2:
        y=i+1
        word=inverted_index.get(w[y])
        result=result.intersection_update(word)
    elif s=="or" and i<=2 :
        x=i-1;
        y=i+1;
        word1=inverted_index.get(w[x])
        word2=inverted_index.get(w[y])
        result=word1.intersection(word2)
    elif s=="or" and i>2:
        y=i+1
        word=inverted_index.get(w[y])
        result=result.union(word)
    elif s=="not" and i>2:
        result=list_docs.difference(result)
    elif s=="not":
        word1=inverted_index.get(w[i])
        result=list_docs.difference(result)
    else:
        continue
print(result)